In [1]:
#we create a simple 2darray.
#this is just to avoid loading numpy :)

#Actions #just some alias
North = 0
East  = 1
South = 2
West  = 3
    
#8puzzle
class PuzzleN :
    def __init__(self, start_config, rows=3,columns=3) : #start_config is a list with all numbers and 0
        self.rows, self.columns = rows, columns
        self.data  = tuple(start_config) #never modified
        self.blank = self.data.index(0)
        self.all_actions = [North, East, South, West]
        self.terminal = (1,2,3,4,5,6,7,8,0)
        
    def actions(self) :
        """find the actions possible"""
        #we assume we are moving the blank to North, South, East, West if possible
        return [ a for a in self.all_actions if self.can_move(a) ]
    
    def move(self, action) :
        """make the move and return new class. The current class is not modified"""
        assert self.can_move(action) #Illegal Move: Cannot make this move.
        row, col = self.row_col(self.blank)
        if action == North :
            row -= 1
        elif action == South :
            row += 1
        elif action == East :
            col += 1
        elif action == West :
            col -= 1
        new_blank = self.position(row,col)
        #print (row,col)
        #print(new_blank)
        #we need to put zero at this new location and put the old value tot he current blank
        new_data = list(self.data)
        new_data[ self.blank ] = new_data[ new_blank ]
        new_data[ new_blank ]  = 0
        return PuzzleN(new_data, self.rows, self.columns)
    
    def is_terminal(self) :
        if self.data == self.terminal :
            return True
        return False
    
    def distance(self) :
        score = 0
        for i, x in enumerate(self.data):
            if x != self.terminal[i]:
                score += 1
        return score
    
    def children(self) : #reurns new states of all legal actions
        act = self.actions()
        if not act :
#             return None, None, None
#         return [ (self.move(a), a, 1) for a in act ]
            return None, None
        return [ (self.move(a), a) for a in act ]
    
    def can_move(self, action) :
        """return true if the 0 can take the action"""
        row, col = self.row_col(self.blank)
        #print(row,col)
        if action is North and row == 0 :           #first row, cannot move north
            return False
        if action is South and row == self.rows-1 : #last row, cannot move south
            return False        
        if action is East and col == self.columns-1:#last column, cannot move East
            return False
        if action is West and col == 0 :            #first column, cannot move West
            return False
        return True
        
    #some utility funtions. We dont need them if using numpy
    def row_col(self, i) :
        row = i // self.columns # integer divisor
        col = i % self.columns #int reminder
        return row, col
    
    def position(self, row, col) :
        return row * self.columns + col
    
    def print(self) :
        for i in range(self.rows * self.columns) :
            print(self.data[i], end='')
            if (i+1) % self.columns == 0 :
                print("")
        print('--------')
        
    def __hash__(self):
        return hash(self.data)
    
    def __eq__(self, other):
        return self.data == other.data
        
#     def __repr__(self):
#         return str(self.data)
        
###################################################################################


#some test
def str_to_list(s) :
    #takes a string of ints and returns the list
    return [ int(i) for i in s.split() ]
s = """
    1 3 7
    2 5 6
    8 0 4
    """
p = PuzzleN(str_to_list(s))
p.print()
q = p.move(North)
q.print()

137
256
804
--------
137
206
854
--------


In [ ]:
from graphs import Pqueue, find_path

In [2]:
def while_bfs_puzzle(start):
    queue = [start]
    visited = [start]
    parent = {}
    while queue:
        current = queue.pop(0) 
        if current.is_terminal():
            break
        for child, action in current.children():
            if child not in visited:
                queue.append(child)
                visited.append(child)
                print(len(visited))
                parent[child.data] = current.data
    print(parent)
    return find_path(parent, current.data)
impossible = [0,2,1,3,4,5,6,7,8]
start = PuzzleN([1,2,3,4,5,6,7,0,8])
# while_bfs_puzzle(start) 
start.distance()

2

In [3]:
#A hacky way to create a random initial 8puzzle
import random
def shuffle_puzzle8(N=10) :
    a = PuzzleN([1,2,3,4,5,6,7,8,0])
    a.move(West).move(North)
    for _ in range(N) :
        act = a.actions()
        #print(act)
        if act:
            a = a.move(act[random.randint(0,len(act)-1)])
    return a

In [7]:
def a_star_search(start):
    queue = Pqueue([(start,0)])
    visited = [start]
    parent = {}
    counter = 0
    while queue:
        current, pcost = queue.pop()
        if current.is_terminal():
            break
        for child, action in current.children():
            path_cost = child.distance() + pcost
            if child in visited and queue.query(child) and queue.query(child)[0] > path_cost:
                print('case 1')
                queue.append(child, path_cost)
                parent[child.data] = current.data
            elif child not in visited and child.data != start.data:
                queue.append(child, path_cost)
                visited.append(child)
                parent[child.data] = current.data
    return find_path(parent, current.data)
        
a = shuffle_puzzle8(5)
a.print()
a_star_search(PuzzleN([1, 2, 3, 4, 8, 0, 7, 6, 5]))

123
460
758
--------


[(1, 2, 3, 4, 8, 0, 7, 6, 5),
 (1, 2, 3, 4, 8, 5, 7, 6, 0),
 (1, 2, 3, 4, 8, 5, 7, 0, 6),
 (1, 2, 3, 4, 0, 5, 7, 8, 6),
 (1, 2, 3, 4, 5, 0, 7, 8, 6),
 (1, 2, 3, 4, 5, 6, 7, 8, 0)]

In [ ]:
a_star_search(shuffle_puzzle8(100))

In [ ]:
#Check if the puzzle is solvable
def getInvCount(arr): 
    inv_count = 0; 
    for i in range(3):
        for j in range(3):
            if (arr[j][i] > 0) & (arr[j][i] > 0) & (arr[j][i] > arr[i][j]):
                inv_count += 1 
    return inv_count 
def isSolvable(puzzle):
    invCount = getInvCount(puzzle) 
    return (invCount % 2 == 0)
puzzle = [[4,8,7],[5,6,3],[1,2,0]]
isSolvable(puzzle)